In [ ]:
import mbuild as mb

In [ ]:
pdb_file = '/data/yang_lab/nehilpkd/biomatsims/smd/solenoid/workspace/c689d3258da041a1dddc343d87c11dd4/0_preprocess/1m8n_cut.pdb'
compound = mb.load(pdb_file)
compound.visualize()

In [ ]:
import mbuild as mb

def extend_beta_solenoid(original_solenoid, c_oxygen, n_hydrogen=None, fibril_axis = 2):
    """Extend the length of a beta solenoid protein."""
    # Create a copy of the solenoid and remove N terminal
    extended_solenoid = mb.clone(original_solenoid)
    if n_hydrogen is not None:
        extended_solenoid.remove(extended_solenoid[n_hydrogen])

    # Remove C terminal from the original solenoid
    original_solenoid.remove(original_solenoid[c_oxygen])

    # Add 'up' and 'down' ports to the removed terminals
    ## Find CA atom of the C terminal
    CA = [atom for atom in list(original_solenoid.particles())[:20] if atom.name == 'CA'][0]
    nitro = [atom for atom in list(extended_solenoid.particles())[:20] if atom.name == 'N'][-1]
    ## Find CA atom of the N terminal
    down_port = mb.Port(anchor=CA, orientation=[0, 0, 1], separation=0.1)
    up_port = mb.Port(anchor=nitro, orientation=[0, 0, -1], separation=0.1)

    # Translate the cloned solenoid to bring N terminal in contact with C terminal
    # shift_vector = nitro.pos - CA.pos

    # shift_vector = shift_vector*3
    shift_vector = CA.pos - nitro.pos
    print(CA.pos, nitro.pos, shift_vector)
    extended_solenoid.translate(shift_vector)

    # Add ports to the cloned solenoid
    original_solenoid.add(up_port, 'up')
    extended_solenoid.add(down_port, 'down')

    original_solenoid.add(extended_solenoid, 'extended_solenoid')


    # Bond the two proteins into a single compound using force_overlap
    mb.force_overlap(original_solenoid,  original_solenoid['up'], extended_solenoid['down'])

    return original_solenoid

# Load the solenoid
solenoid = mb.load(pdb_file)
extend_beta_solenoid(solenoid, c_oxygen = 0).visualize()

In [ ]:
solenoid = mb.load(pdb_file)
# solenoid[-2].name = 'S'
solenoid.translate_to([0, 0, 0])
copy = mb.clone(solenoid)
copy.translate([0, 3, 0])
new_copy = mb.clone(solenoid)
new_copy.translate([0, 6, 0])
solenoid.add(copy)
solenoid.add(new_copy) 
solenoid.visualize()